## 📚 Prerequisites

Ensure that your Azure Services are properly set up, your Conda environment is created, and your environment variables are configured as per the instructions in the [SETTINGS.md](SETTINGS.md) file.

In [1]:
import os

# Define the target directory
target_directory = (
    r"C:\Users\pablosal\Desktop\gbbai-azure-aoai-faq"  # change your directory here
)

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-aoai-faq


## 📚 Prerequisites

Ensure that your Azure Services are properly set up, your Conda environment is created, and your environment variables are configured as per the instructions in the [SETTINGS.md](SETTINGS.md) file.

## 📋 Table of Contents

This notebook assists in testing and retrieving the headers of Azure OpenAI, covering the following sections:

1. [**Setting Up Azure OpenAI Client**](#setting-up-azure-openai-client): Outlines the process of initializing the Azure OpenAI client.

2. [**Calling Azure OpenAI API**](#calling-azure-openai-api): Discusses how to make API calls to Azure OpenAI.

3. [**Extracting Headers and Payload Metadata**](#extracting-headers-and-payload-metadata): Explores how to extract headers and payload metadata from the API response.

4. [**Analyzing Rate Limit Info**](#analyzing-rate-limit-info): Details the steps to analyze the rate limit information from the API response.

For additional information, refer to the following resources:
- [AOAI Assistants API Documentation](https://learn.microsoft.com/en-us/azure/ai-services/openai/assistants-reference?tabs=python)

In [2]:
from dotenv import load_dotenv
from utils.ml_logging import get_logger

# Set up logger
logger = get_logger()

# Load environment variables from .env file
load_dotenv()

True

In [3]:
from src.aoai.azure_assistants import AzureOpenAssistantManager

# Create an instance of the client. You can find it in src/aoai/azure_openai.py.
# It is essentially a wrapper using dependency injection to automate the initialization
# and most used API calls.
api_key = os.getenv("AZURE_AOAI_KEY_ASSISTANTS")
api_version = os.getenv("AZURE_AOAI_API_VERSION_ASSISTANTS")
azure_endpoint = os.getenv("AZURE_AOAI_API_ENDPOINT_ASSISTANTS")
completion_model_name = os.getenv("AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID_ASSISTANTS")

#load client
azure_openai_client = AzureOpenAssistantManager(api_key, api_version, azure_endpoint, completion_model_name=completion_model_name)

## Add files

In [4]:
## create files

files = azure_openai_client.openai_client.files.create(
    file=open(r"C:\Users\pablosal\Desktop\gbbai-azure-ai-aoai\utils\artifacts\test.txt", "rb"),
    purpose="assistants"
)

In [5]:
files.id

'assistant-FeE5GxxRreGQdb3oo2v8ELE5'

In [6]:
# Create an assistant
assistant = azure_openai_client.openai_client.beta.assistants.create(
    name="Math assistant",
    instructions=f"You are a helpful AI assistant who can write code to help answer math questions." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a math questions you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the result."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model=f'{completion_model_name}',
    file_ids=[files.id]
)

logger.info(assistant.model_dump_json(indent=2))

2024-02-29 13:26:53,523 - micro - MainProcess - INFO     {
  "id": "asst_onwwgCjaMJq8AmLj2yDCxsEi",
  "created_at": 1709234814,
  "description": null,
  "file_ids": [
    "assistant-FeE5GxxRreGQdb3oo2v8ELE5"
  ],
  "instructions": "You are a helpful AI assistant who can write code to help answer math questions.You have access to a sandboxed environment for writing and testing code.When you are asked to create a math questions you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the result.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4",
  "name": "Math assistant",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
} (2520778088.py:<module>:17)


In [7]:
# Create a thread
thread = azure_openai_client.openai_client.beta.threads.create()
logger.info(thread)

2024-02-29 13:26:58,473 - micro - MainProcess - INFO     Thread(id='thread_rBzRbGybJcyvxW9TIaVvyMXk', created_at=1709234819, metadata={}, object='thread') (3556306472.py:<module>:3)


In [8]:
# Add a user question to the thread
message = azure_openai_client.openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    file_ids=
    [
        files.id
    ],
    content="If a topic modeling algorithm identifies 5 topics in a 10,000-word document, what is the average number of words per topic? (Answer: 10,000 words / 5 topics = 2000 words per topic)."
)

In [9]:
run = azure_openai_client.openai_client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [10]:
import time
from IPython.display import clear_output

COMPLETED_STATUSES = ["completed", "cancelled", "expired", "failed"]
SLEEP_TIME = 5

def print_elapsed_time(start_time):
    elapsed_time = time.time() - start_time
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    logger.info(f"Elapsed time: {minutes} minutes {seconds} seconds")

start_time = time.time()

run_status = run.status

while run_status not in COMPLETED_STATUSES:
    time.sleep(SLEEP_TIME)
    run = azure_openai_client.openai_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    run_status = run.status
    print_elapsed_time(start_time)
    logger.info(f'Status: {run_status}')
    clear_output(wait=True)

messages = azure_openai_client.openai_client.beta.threads.messages.list(thread_id=thread.id)

logger.info(f'Status: {run_status}')
print_elapsed_time(start_time)
logger.info(messages.model_dump_json(indent=2))

2024-02-29 13:27:35,250 - micro - MainProcess - INFO     Status: completed (1685151657.py:<module>:27)
2024-02-29 13:27:35,251 - micro - MainProcess - INFO     Elapsed time: 0 minutes 22 seconds (1685151657.py:print_elapsed_time:11)
2024-02-29 13:27:35,252 - micro - MainProcess - INFO     {
  "data": [
    {
      "id": "msg_OlGwCCoIGQejHsigfYfngUIw",
      "assistant_id": "asst_onwwgCjaMJq8AmLj2yDCxsEi",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The average number of words per topic in a 10,000-word document with 5 topics is 2,000 words per topic."
          },
          "type": "text"
        }
      ],
      "created_at": 1709234852,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_t7G9ltFKzbZixbhdPkNQe8iL",
      "thread_id": "thread_rBzRbGybJcyvxW9TIaVvyMXk"
    },
    {
      "id": "msg_hw2aV8ZsFOqIpaJ9S7QAGcoJ",
      "assistant_id": "asst_onww

In [11]:
import json

data = json.loads(messages.model_dump_json(indent=2))
logger.info(data['data'][0]['content'][0]['text']['value'])

2024-02-29 13:27:35,267 - micro - MainProcess - INFO     The average number of words per topic in a 10,000-word document with 5 topics is 2,000 words per topic. (1105317613.py:<module>:4)
